# Data Reading

In [1]:
import pandas as pd

In [2]:
def load_data(prefix):
    def fix_path(src):
        return prefix + str('/IMG/') +src.split('/')[-1]    
    
    data = pd.read_csv( prefix + str('/driving_log.csv'), header=None, skiprows=1, names=[
        'center_img_path', 
        'left_img_path',
        'right_img_path',
        'steering_angle',
        'throttle',
        'break',
        'speed'
    ])
    
    data['center_img_path'] =  data['center_img_path'].apply(fix_path)
    data['left_img_path'] =  data['left_img_path'].apply(fix_path)
    data['right_img_path'] =  data['right_img_path'].apply(fix_path)

    return data

In [3]:
datasets = [load_data(prefix) for prefix in  [ './data/udacity', './data/train', './data/train_2'] ]

In [4]:
data = pd.concat( datasets )
data.head(3)

,center_img_path,left_img_path,right_img_path,steering_angle,throttle,break,speed
0,./data/udacity/IMG/center_2016_12_01_13_30_48_...,./data/udacity/IMG/left_2016_12_01_13_30_48_28...,./data/udacity/IMG/right_2016_12_01_13_30_48_2...,0.0,0.0,0.0,22.148290
1,./data/udacity/IMG/center_2016_12_01_13_30_48_...,./data/udacity/IMG/left_2016_12_01_13_30_48_40...,./data/udacity/IMG/right_2016_12_01_13_30_48_4...,0.0,0.0,0.0,21.879630
2,./data/udacity/IMG/center_2016_12_01_13_31_12_...,./data/udacity/IMG/left_2016_12_01_13_31_12_93...,./data/udacity/IMG/right_2016_12_01_13_31_12_9...,0.0,0.0,0.0,1.453011


# Data Generation

In [5]:
import pandas as pd
import numpy as np
import cv2

def unison_shuffle(a, b):
    assert len(a) == len(b)
    p = np.random.permutation(len(a))
    return a[p], b[p]


def generator(batch_size, data):
    def load_img(src):
        return cv2.imread(src)

    correction = 0.3
    while True:
        subsample = data.sample(batch_size)
        
        subsample['center_img'] = subsample.center_img_path.apply(load_img)
        subsample['left_img'] = subsample.left_img_path.apply(load_img)
        subsample['right_img'] = subsample.right_img_path.apply(load_img)

        X_center = np.asarray( subsample.center_img.values.tolist() )
        y_center = subsample[['steering_angle']].values
        
        X_center_flipped = np.fliplr(X_center)
        y_center_flipped = subsample[['steering_angle']].values * (-1)

        X_left = np.asarray( subsample.left_img.values.tolist() )
        y_left = subsample[['steering_angle']].values + correction

        X_right = np.asarray( subsample.right_img.values.tolist() )
        y_right = subsample[['steering_angle']].values - correction

        X = np.concatenate( [X_center, X_center_flipped], axis=0)
        y = np.concatenate( [y_center, y_center_flipped], axis=0)
        
        yield unison_shuffle(X, y)

In [6]:
validation_data = data.sample( int(data.shape[0] * 0.1) )
train_data = data.drop(validation_data.index)

#X_valid, y_valid = next(generator(validation_data.shape[0], validation_data))

# Neural Net

In [7]:
import os

In [8]:
def preprocess_image(img):
    from keras.backend import tf as ktf
    import keras.backend as K
    
    ktf.image.resize_images(img, (80, 160))
    return (K.identity(img) / 255.0) - 0.5


In [9]:
import keras
from keras.models import Sequential
from keras.layers import Conv2D, Flatten, Dense, Lambda, Cropping2D, Dropout, Activation, Conv2D
from keras.applications import VGG16
from keras.regularizers import l2

tb = keras.callbacks.TensorBoard(log_dir='./logs')

model = Sequential()
model.add(Cropping2D(cropping=((50,20), (0,0)), input_shape=(160,320,3)))
model.add(Lambda(preprocess_image))
model.add(Conv2D(24, (5, 5),strides=(2,2),activation="relu"))
model.add(Conv2D(36, (5, 5),strides=(2,2),activation="relu"))
model.add(Conv2D(48, (5, 5),strides=(2,2),activation="relu"))
model.add(Conv2D(64, (3, 3),activation="relu"))
model.add(Conv2D(64, (3, 3),activation="relu"))
model.add(Flatten())
model.add(Dense(100))
model.add(Dense(50))
model.add(Dense(10))
model.add(Dense(1))

model.compile( optimizer='adam', loss='mse' )
model.summary()

Using TensorFlow backend.


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
cropping2d_1 (Cropping2D)    (None, 90, 320, 3)        0         
_________________________________________________________________
lambda_1 (Lambda)            (None, 90, 320, 3)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 43, 158, 24)       1824      
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 20, 77, 36)        21636     
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 8, 37, 48)         43248     
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 6, 35, 64)         27712     
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 4, 33, 64)         36928     
__________

In [10]:
batch_size=64
epochs=15
steps_per_epoch = train_data.shape[0] // batch_size 

model.fit_generator( generator=generator(batch_size, train_data), 
                    steps_per_epoch=steps_per_epoch, 
                    epochs=epochs, 
                    callbacks=[tb],
                    validation_data=generator(batch_size, validation_data),
                    validation_steps=steps_per_epoch // 10
                   )


Epoch 1/15
171/171 [==============================] - 68s - loss: 0.0230 - val_loss: 0.0144
Epoch 2/15
171/171 [==============================] - 65s - loss: 0.0132 - val_loss: 0.0154
Epoch 3/15
171/171 [==============================] - 66s - loss: 0.0120 - val_loss: 0.0117
Epoch 4/15
171/171 [==============================] - 66s - loss: 0.0115 - val_loss: 0.0132
Epoch 5/15
171/171 [==============================] - 65s - loss: 0.0116 - val_loss: 0.0108
Epoch 6/15
171/171 [==============================] - 64s - loss: 0.0117 - val_loss: 0.0097
Epoch 7/15
171/171 [==============================] - 65s - loss: 0.0106 - val_loss: 0.0133
Epoch 8/15
171/171 [==============================] - 65s - loss: 0.0103 - val_loss: 0.0129
Epoch 9/15
171/171 [==============================] - 65s - loss: 0.0101 - val_loss: 0.0097
Epoch 10/15
171/171 [==============================] - 64s - loss: 0.0096 - val_loss: 0.0094
Epoch 11/15
171/171 [==============================] - 65s - loss: 0.0093 - val

In [11]:
model.save( '2017-07-29.h5' )